# UCI Adult Dataset Process Demo.
Only demonstrate the basic useage of this dataset and some algorithms in scikit library(And most copy with minimum changes from [【机器学习实验】scikit-learn的主要模块和基本使用](http://www.jianshu.com/p/1c6efdbce226)), if we want to have better and more meaningful results,we should do some more work.

## load the preprocessing: transform description to number([male,female]->[0,1])

In [1]:
def loadXandY(fileName):
    import numpy as np
    
    # load the CSV file as a numpy matrix
    with open(fileName, 'r') as f:
        dataset = np.loadtxt(f, delimiter=",")
    
    # separate the data from the target attributes
    attrsize = len(dataset[0])
    
    print(len(dataset), attrsize)
    
    X = dataset[:,0:-1]
    y = dataset[:,-1]
    return X,y

In [2]:
baseDir = 'H:/practice/scikit_class/scikit_learning/uci_adult/adult_data/'
# baseDir = 'adult_data/'

fileName = baseDir+'adult.data.num'

testFileName = baseDir+'adult.test.num'

X,y = loadXandY(fileName)
TX,Ty = loadXandY(testFileName)

32561 15
16281 15


# deal with the missing value
[reference](http://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values)

In [3]:
def simpleImputer(X):
    from sklearn.preprocessing import Imputer
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    imp.fit(X)
    return imp.transform(X)

In [4]:
X = simpleImputer(X)
TX = simpleImputer(TX)

数据归一化(Data Normalization)

In [5]:
def simpleProcessing(X):
    from sklearn import preprocessing
    # normalize the data attributes
    normalized_X = preprocessing.normalize(X)
    # standardize the data attributes
    standardized_X = preprocessing.scale(X)
    return standardized_X

In [6]:
X = simpleProcessing(X)
TX = simpleProcessing(TX)

特征选择(Feature Selection)

In [7]:
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(X, y)
# display the relative importance of each attribute
print(model.feature_importances_)

[ 0.16746732  0.04252195  0.16567563  0.03698216  0.08049178  0.10211605
  0.07715989  0.04817133  0.01456404  0.03066858  0.09105681  0.03045428
  0.09249326  0.02017692]


In [8]:
def show_predictions(y, predicted):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()
    ax.scatter(y, predicted)
    ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    plt.show()

In [9]:
def make_predictions(model, X, y):
    from sklearn import metrics
    # make predictions
    expected = y
    predicted = model.predict(X)
    # summarize the fit of the model
    print(metrics.classification_report(expected, predicted))
    print(metrics.confusion_matrix(expected, predicted))
    print('\n')
#     show_predictions(expected[::len(expected)//20], predicted[::len(predicted)//20])

In [10]:
def cross_validation(model, X, y):
    from sklearn.model_selection import KFold, cross_val_score
    k_fold = KFold(n_splits=3)
    return cross_val_score(model, X, y, cv=k_fold, n_jobs=-1)

In [11]:
# 逻辑回归
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X, y)
print(model)

make_predictions(model, X, y)

make_predictions(model, TX, Ty)

print(cross_validation(model, X, y))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
             precision    recall  f1-score   support

        0.0       0.73      0.55      0.63      7841
        1.0       0.87      0.93      0.90     24720

avg / total       0.83      0.84      0.83     32561

[[ 4304  3537]
 [ 1609 23111]]


             precision    recall  f1-score   support

        0.0       0.72      0.54      0.62      3846
        1.0       0.87      0.94      0.90     12435

avg / total       0.83      0.84      0.83     16281

[[ 2094  1752]
 [  805 11630]]


[ 0.83876912  0.84116455  0.84391413]


In [12]:
# 朴素贝叶斯
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X, y)
print(model)

make_predictions(model, X, y)

make_predictions(model, TX, Ty)

print(cross_validation(model, X, y))

GaussianNB(priors=None)
             precision    recall  f1-score   support

        0.0       0.69      0.40      0.51      7841
        1.0       0.83      0.94      0.88     24720

avg / total       0.80      0.81      0.79     32561

[[ 3160  4681]
 [ 1407 23313]]


             precision    recall  f1-score   support

        0.0       0.69      0.41      0.51      3846
        1.0       0.84      0.94      0.89     12435

avg / total       0.80      0.82      0.80     16281

[[ 1567  2279]
 [  707 11728]]


[ 0.8115902   0.81241938  0.81608772]


In [13]:
# K近邻
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
# fit a k-nearest neighbor model to the data
model = KNeighborsClassifier()
model.fit(X, y)
print(model)

make_predictions(model, X, y)

make_predictions(model, TX, Ty)

print(cross_validation(model, X, y))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
             precision    recall  f1-score   support

        0.0       0.78      0.69      0.74      7841
        1.0       0.91      0.94      0.92     24720

avg / total       0.88      0.88      0.88     32561

[[ 5437  2404]
 [ 1504 23216]]


             precision    recall  f1-score   support

        0.0       0.66      0.58      0.62      3846
        1.0       0.88      0.91      0.89     12435

avg / total       0.83      0.83      0.83     16281

[[ 2241  1605]
 [ 1134 11301]]


[ 0.82623917  0.8291874   0.83331798]


In [14]:
# 决策树
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
# fit a CART model to the data
model = DecisionTreeClassifier()
model.fit(X, y)
print(model)

make_predictions(model, X, y)

make_predictions(model, TX, Ty)


print(cross_validation(model, X, y))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      7841
        1.0       1.00      1.00      1.00     24720

avg / total       1.00      1.00      1.00     32561

[[ 7841     0]
 [    1 24719]]


             precision    recall  f1-score   support

        0.0       0.61      0.61      0.61      3846
        1.0       0.88      0.88      0.88     12435

avg / total       0.81      0.81      0.81     16281

[[ 2340  1506]
 [ 1526 10909]]


[ 0.80652294  0.81463055  0.81295494]


In [15]:
# This block is still about `决策树`

# The train set have only one outlier, so it's clearly overfit(since the test dataset is not so appealing),
# we want to have this outlier's info,since it can be interesting
import numpy as np
expected = y
predicted = model.predict(X)
diff = expected - predicted
diff_location = list(np.where(diff != 0)[0])
print(len(diff_location), diff_location)

def getLine(fileName, lineNum):
    with open(fileName, 'r') as f:
        all = f.readlines()
    
    return all[lineNum].strip()

getLine(baseDir+'adult.data', 848)

1 [848]


'39, Private, 138192, Bachelors, 13, Married-civ-spouse, Craft-repair, Husband, White, Male, 0, 0, 40, United-States, <=50K'

In [16]:
# SVM
from sklearn import metrics
from sklearn.svm import SVC
# fit a SVM model to the data
model = SVC()
model.fit(X, y)
print(model)


make_predictions(model, X, y)

make_predictions(model, TX, Ty)

print(cross_validation(model, X, y))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
             precision    recall  f1-score   support

        0.0       0.78      0.56      0.65      7841
        1.0       0.87      0.95      0.91     24720

avg / total       0.85      0.85      0.85     32561

[[ 4366  3475]
 [ 1247 23473]]


             precision    recall  f1-score   support

        0.0       0.76      0.54      0.63      3846
        1.0       0.87      0.95      0.91     12435

avg / total       0.84      0.85      0.84     16281

[[ 2089  1757]
 [  666 11769]]


[ 0.84567901  0.85019348  0.84953469]


In [17]:
# 优化算法参数
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.grid_search import GridSearchCV
# prepare a range of alpha values to test
alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
# create and fit a ridge regression model, testing each alpha
model = Ridge()
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas))
grid.fit(X, y)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_.alpha)

H:\py_env\scikit_sys_site_packages\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
H:\py_env\scikit_sys_site_packages\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


GridSearchCV(cv=None, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([  1.00000e+00,   1.00000e-01,   1.00000e-02,   1.00000e-03,
         1.00000e-04,   0.00000e+00])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)
0.31229463325545104
1.0


In [18]:
import numpy as np
from scipy.stats import uniform as sp_rand
from sklearn.linear_model import Ridge
from sklearn.grid_search import RandomizedSearchCV
# prepare a uniform distribution to sample for the alpha parameter
param_grid = {'alpha': sp_rand()}
# create and fit a ridge regression model, testing random alpha values
model = Ridge()
rsearch = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=100)
rsearch.fit(X, y)
print(rsearch)
# summarize the results of the random parameter search
print(rsearch.best_score_)
print(rsearch.best_estimator_.alpha)

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
          fit_params={}, iid=True, n_iter=100, n_jobs=1,
          param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001A213A70400>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring=None, verbose=0)
0.31229463315387784
0.996920839648
